In [1]:
import os

import jax
import jax.numpy as jnp
import numpy as np
from scipy import optimize

from vibrojet.basis_utils import ContrBasis, HermiteBasis, generate_prod_ind
from vibrojet.keo import Gmat, com
from vibrojet.potentials import h2co_AYTY
from vibrojet.taylor import deriv_list

jax.config.update("jax_enable_x64", True)

Equilibrium geometry

In [2]:
vmin = optimize.minimize(h2co_AYTY.poten, [1, 1, 1, 2, 2, np.pi])
r0 = vmin.x
v0 = vmin.fun
print("equilibrium coordinates:", r0)
print("min of the potential:", v0)

equilibrium coordinates: [1.19999999 1.1        1.1        2.13       2.13       3.14159264]
min of the potential: 1.735341153700765e-11


Coordinate mapping

In [3]:
masses = [12.0, 15.990526, 1.00782505, 1.00782505]  # masses of C, O, H, H

ncoo = len(r0)


# @eckart(r0, masses)
@com(masses)
def internal_to_cartesian(internal_coords):
    rCO, rCH1, rCH2, aOCH1, aOCH2, tau = internal_coords
    xyz = [
        [0.0, 0.0, 0.0],
        [0.0, 0.0, rCO],
        [
            rCH1 * jnp.sin(aOCH1) * jnp.cos(tau * 0.5),
            -rCH1 * jnp.sin(aOCH1) * jnp.sin(tau * 0.5),
            rCH1 * jnp.cos(aOCH1),
        ],
        [
            rCH2 * jnp.sin(aOCH2) * jnp.cos(tau * 0.5),
            rCH2 * jnp.sin(aOCH2) * jnp.sin(tau * 0.5),
            rCH2 * jnp.cos(aOCH2),
        ],
    ]
    return jnp.array(xyz, dtype=jnp.float64)

Generate expansions for the KEO and PES

In [4]:
max_pow = 6
powers = [np.arange(max_pow + 1)] * ncoo
deriv_ind, deriv_mind = next(
    generate_prod_ind(powers, select=lambda ind: np.sum(ind) <= max_pow)
)

print("max expansion power:", max_pow)
print("number of expansion terms:", len(deriv_ind))

max expansion power: 6
number of expansion terms: 924


In [5]:
poten_file = "_h2co_poten_coefs.npy"
if os.path.exists(poten_file):
    print(f"load potential expansion coefs from file {poten_file}")
    poten_coefs = np.load(poten_file)
else:
    poten_coefs = deriv_list(h2co_AYTY.poten, deriv_ind, r0, if_taylor=True)
    np.save(poten_file, poten_coefs)

load potential expansion coefs from file _h2co_poten_coefs.npy


In [6]:
Gmat_file = "_h2co_Gmat_coefs.npy"
if os.path.exists(Gmat_file):
    print(f"load G-matrix expansion coefs from file {Gmat_file}")
    Gmat_coefs = np.load(Gmat_file)
else:
    Gmat_coefs = deriv_list(
        lambda x: Gmat(x, masses, internal_to_cartesian), deriv_ind, r0, if_taylor=True
    )
    np.save(Gmat_file, Gmat_coefs)

load G-matrix expansion coefs from file _h2co_Gmat_coefs.npy


Define linear mapping between coordinates of Hermite functions and internal valence coordinates

In [7]:
mask = deriv_ind != 0
ind0 = np.where(mask.sum(axis=1) == 0)[0][0]
mu = np.diag(Gmat_coefs[ind0])[:ncoo]

ind2 = np.array(
    [
        np.where((mask.sum(axis=1) == 1) & (deriv_ind[:, icoo] == 2))[0][0]
        for icoo in range(ncoo)
    ]
)
freq = poten_coefs[ind2] * 2

lin_a = jnp.sqrt(jnp.sqrt(mu / freq))
lin_b = r0

print("x->r linear mapping parameters 'a':", lin_a)
print("x->r linear mapping parameters 'b':", lin_b)

# x->r linear mapping function
x_to_r_map = lambda x, icoo: lin_a[icoo] * x + lin_b[icoo]
jac_x_to_r_map = lambda x, icoo: np.ones_like(x) * lin_a[icoo]

x->r linear mapping parameters 'a': [0.05235088 0.11034365 0.11034365 0.15434588 0.15434588 0.3003904 ]
x->r linear mapping parameters 'b': [1.19999999 1.1        1.1        2.13       2.13       3.14159264]


Define primitive basis

In [8]:
nbas = [10] * ncoo
npoints = [80] * ncoo

p0, p1, p2, p3, p4, p5 = [
    HermiteBasis(
        icoo,
        nbas[icoo],
        npoints[icoo],
        lambda x: x_to_r_map(x, icoo),
        r0[icoo],
        deriv_ind[:, icoo],
    )
    for icoo in range(ncoo)
]

Build contracted basis

In [9]:
# test 1D shit
c0, c1, c2, c3, c4, c5 = [
    ContrBasis(
        (icoo,), (p0, p1, p2, p3, p4, p5), lambda _: True, Gmat_coefs, poten_coefs
    )
    for icoo in range(ncoo)
]

# couple CH1 and CH2
c12 = ContrBasis(
    (1, 2),
    (p0, c1, c2, p3, p4, p5),
    lambda _: True,
    Gmat_coefs,
    poten_coefs,
    emax=20000,
)

# couple H1CO and H2CO
c34 = ContrBasis(
    (3, 4),
    (p0, p1, p2, c3, c4, p5),
    lambda _: True,
    Gmat_coefs,
    poten_coefs,
    emax=20000,
)

# couple all together
p_coefs = np.array([2, 2, 1, 1])
p_max = 6
c = ContrBasis(
    (0, 1, 2, 3),
    (c0, c12, c34, c5),
    lambda ind: np.sum(np.array(ind) * p_coefs) < p_max,
    Gmat_coefs,
    poten_coefs,
)
e = c.enr
print(e[0], e - e[0])

5896.959659374803 [   0.         1089.44177422 1170.56694786 1405.2708966  2094.09954829
 2208.15711713 2299.44508092 2357.20088939 2500.54497363 2582.85224961
 2790.8613031  2965.2738412  3111.50973988 3185.20854927 3284.203136
 3322.0587227  3433.62520039 3543.560982   3549.25597141 3788.39499255
 3878.09886886 3923.97124758 3950.31394818 3953.59753604 4019.3720713
 4078.55938002 4125.76979541 4268.49802008 4317.07340424 4420.52497405
 4509.66817599 4634.14721163 4679.34197841 4784.27775602 4870.01118546
 4881.58856067 5079.07542926 5118.31673326 5129.92606184 5144.67581563
 5177.75101619 5321.49358002 5419.05264154 5703.87345504 5745.81548523
 6050.1521393  6059.90222099 6097.56538997 6319.0045481  6332.94008466]
